In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
%%bash
#This is a bash cell 
wget -O - https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz | aws s3 cp - s3://redfin-raw-data-s3bucket/city_market_tracker.tsv000.gz

--2024-09-29 11:00:08--  https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz
Resolving redfin-public-data.s3.us-west-2.amazonaws.com (redfin-public-data.s3.us-west-2.amazonaws.com)... 52.92.192.122, 3.5.76.178, 3.5.76.242, ...
Connecting to redfin-public-data.s3.us-west-2.amazonaws.com (redfin-public-data.s3.us-west-2.amazonaws.com)|52.92.192.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1297212322 (1.2G) [application/x-www-form-urlencoded]
Saving to: ‘STDOUT’

     0K .......... .......... .......... .......... ..........  0%  323K 65m24s
    50K .......... .......... .......... .......... ..........  0%  323K 65m23s
   100K .......... .......... .......... .......... ..........  0% 86.2M 43m40s
   150K .......... .......... .......... .......... ..........  0%  324K 49m1s
   200K .......... .......... .......... .......... ..........  0%  199M 39m14s
   250K .......... .......... .......... ........

In [12]:
spark = SparkSession.builder.appName("redfinDataAnalysis").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
redfin_data = spark.read.csv("s3://redfin-raw-data-s3bucket/city_market_tracker.tsv000.gz", header=True, inferSchema=True, sep= "\t")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
redfin_data.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------+---------------+-----------+--------------+--------+----------------------+-------------+---------+----------+----------+--------------------+----------------+-----------------+---------------------+---------------------+-----------------+---------------------+---------------------+------------------+-------------------+-------------------+------------------+--------------------+--------------------+----------+--------------------+--------------------+-------------+--------------------+--------------------+------------+--------------------+--------------------+---------+--------------------+-------------------+----------------+--------------------+--------------------+----------+--------------+--------------+------------------+--------------------+--------------------+------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-----------------------+---------------------------+---------------------

In [17]:
redfin_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- period_begin: date (nullable = true)
 |-- period_end: date (nullable = true)
 |-- period_duration: integer (nullable = true)
 |-- region_type: string (nullable = true)
 |-- region_type_id: integer (nullable = true)
 |-- table_id: integer (nullable = true)
 |-- is_seasonally_adjusted: string (nullable = true)
 |-- region: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- property_type_id: integer (nullable = true)
 |-- median_sale_price: double (nullable = true)
 |-- median_sale_price_mom: double (nullable = true)
 |-- median_sale_price_yoy: double (nullable = true)
 |-- median_list_price: double (nullable = true)
 |-- median_list_price_mom: double (nullable = true)
 |-- median_list_price_yoy: double (nullable = true)
 |-- median_ppsf: double (nullable = true)
 |-- median_ppsf_mom: double (nullable = true)
 |-- median_ppsf_yoy: double (nul

In [19]:
redfin_data.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['period_begin', 'period_end', 'period_duration', 'region_type', 'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region', 'city', 'state', 'state_code', 'property_type', 'property_type_id', 'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy', 'median_list_price', 'median_list_price_mom', 'median_list_price_yoy', 'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf', 'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold', 'homes_sold_mom', 'homes_sold_yoy', 'pending_sales', 'pending_sales_mom', 'pending_sales_yoy', 'new_listings', 'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom', 'inventory_yoy', 'months_of_supply', 'months_of_supply_mom', 'months_of_supply_yoy', 'median_dom', 'median_dom_mom', 'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom', 'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom', 'sold_above_list_yoy', 'price_drops', 'price_drops_mom', 'price_drops_yoy', 'off_market_in_two_wee

In [26]:
# specifying the columns i need

df_redfin = redfin_data.select(['period_end','period_duration', 'city', 'state', 'property_type',
    'median_sale_price', 'median_ppsf', 'homes_sold', 'inventory', 'months_of_supply', 'median_dom', 'sold_above_list', 'last_updated'])
df_redfin.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+----------+----------+--------------------+-----------------+------------------+----------+---------+----------------+----------+-------------------+-------------------+
|period_end|period_duration|      city|     state|       property_type|median_sale_price|       median_ppsf|homes_sold|inventory|months_of_supply|median_dom|    sold_above_list|       last_updated|
+----------+---------------+----------+----------+--------------------+-----------------+------------------+----------+---------+----------------+----------+-------------------+-------------------+
|2017-09-30|             30|   Chicago|  Illinois|Multi-Family (2-4...|         259500.0|107.17509025270758|       284|     1316|             4.6|        42| 0.2711267605633803|2024-09-18 14:24:31|
|2023-03-31|             30| Lexington|Washington|     All Residential|         424900.0|210.76388888888889|         3|        4|             1.3|        44| 0.3333333333333333|2024-09-18 14:24:31|
|2020-07-3

In [27]:
df_redfin.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5559476

In [41]:
# Count null values in each column
null_counts = [df_redfin.where(isnull(col_name)).count() for col_name in df_redfin.columns]
null_counts

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0, 0, 0, 0, 6220, 72722, 5798, 448381, 369270, 71498, 37611, 0, 0]

In [ ]:
df_redfin = df_redfin.na.drop()

In [ ]:
null_counts = [df_redfin.where(isnull(col_name)).count() for col_name in df_redfin.columns]
null_counts

In [36]:
#Extract year from period_end and save in a new column "period_end_yr"
df_redfin = df_redfin.withColumn("period_end_yr", year(col("period_end")))

#Extract month from period_end and save in a new column "period_end_month"
df_redfin = df_redfin.withColumn("period_end_month", month(col("period_end")))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
# Drop period_end and last_updated columns
df_redfin = df_redfin.drop("period_end", "last_updated")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df_redfin.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+----------+----------+--------------------+-----------------+------------------+----------+---------+----------------+----------+-------------------+-------------+----------------+
|period_duration|      city|     state|       property_type|median_sale_price|       median_ppsf|homes_sold|inventory|months_of_supply|median_dom|    sold_above_list|period_end_yr|period_end_month|
+---------------+----------+----------+--------------------+-----------------+------------------+----------+---------+----------------+----------+-------------------+-------------+----------------+
|             30|   Chicago|  Illinois|Multi-Family (2-4...|         259500.0|107.17509025270758|       284|     1316|             4.6|        42| 0.2711267605633803|         2017|               9|
|             30| Lexington|Washington|     All Residential|         424900.0|210.76388888888889|         3|        4|             1.3|        44| 0.3333333333333333|         2023|               3|
|         

In [42]:
#let's map the month number to their respective month name.

df_redfin = df_redfin.withColumn("period_end_month", 
                   when(col("period_end_month") == 1, "January")
                   .when(col("period_end_month") == 2, "February")
                   .when(col("period_end_month") == 3, "March")
                   .when(col("period_end_month") == 4, "April")
                   .when(col("period_end_month") == 5, "May")
                   .when(col("period_end_month") == 6, "June")
                   .when(col("period_end_month") == 7, "July")
                   .when(col("period_end_month") == 8, "August")
                   .when(col("period_end_month") == 9, "September")
                   .when(col("period_end_month") == 10, "October")
                   .when(col("period_end_month") == 11, "November")
                   .when(col("period_end_month") == 12, "December")
                   .otherwise("Unknown")
                 )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
#let us write the final dataframe into our s3 bucket as a parquet file.
s3_bucket = "s3://redfin-transformed-data-s3bucket/redfin_data.parquet"
df_redfin.write.mode("overwrite").parquet(s3_bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…